In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=9df52e743361359703f57d1b17e76e7d95f3d9bd2fddd5e0c7d20dfa47c27474
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Problema 2:

In [4]:
# Definimos el nodo fuente y las aristas
source_node = 1
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

# RDD de las aristas
edges_rdd = sc.parallelize(edges)

# Obtenemos los nodos únicos del RDD de aristas
nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct()

In [5]:
nodes.collect()

[2, 4, 1, 3]

In [6]:
# Función que inicializa el costo del nodo inicial y el resto de nodos
def initialize(node):
  if node == source_node:
    return (node, 0)
  else:
    return (node, float('inf'))

In [7]:
# Función que genera mensajes
def generate_messages_2(edge):
  src, dest, cost = edge

  for node in current_costs_dict:
    if node == src:
      return (dest, current_costs_dict[src] + cost)

  return (source_node, 0)
  # return [(dest, current_costs_dict[src] + cost) for node in current_costs_dict if node == src]

# Función que actualiza los costos
def update_costs(node, costs):
  current_cost = float('inf')
  for cost in costs:
    if cost < current_cost:
      current_cost = cost
  return (node, current_cost)

In [8]:
current_costs = nodes.map(initialize)
current_costs.collect()

[(2, inf), (4, inf), (1, 0), (3, inf)]

In [9]:
current_costs_dict = dict(current_costs.collect())
current_costs_dict

{2: inf, 4: inf, 1: 0, 3: inf}

#### En el siguiente "while" loop es donde iteramos el procedimiento anterior hasta que pare de cambiar o hasta que llegue al maximo de iteraciones n.

In [10]:
max_iterations = 10
convergence = 2
actual_iter = 0
no_change = 0

while no_change < convergence and actual_iter < max_iterations:
  previous_costs = current_costs.collectAsMap()
  messages = edges_rdd.map(lambda x: generate_messages_2(x))
  current_costs = current_costs.union(messages).reduceByKey(lambda x, y: min(x, y))
  current_costs_dict = dict(current_costs.collect())

  if previous_costs == current_costs.collectAsMap():
    no_change += 1
  else:
    no_change = 0

  actual_iter += 1

In [11]:
current_costs.collect()

[(1, 0), (2, 10), (3, 13), (4, 34)]

### Ahora probabermos si nuestro algoritmo es escalable por lo que usaremos el grafo "cora" como ejemplo que usamos en el problema 1:

In [14]:
import pandas as pd
import networkx as nx
from random import randint

In [ ]:
# IMPORTANTE: Si el archivo cora.cites esta tiene otra ruta, hay que cambiar la variable RUTA
RUTA = 'cora.cites'
citas = pd.read_csv(RUTA,sep="\t",
    header=None,
    names=["target", "source"])
G = nx.from_pandas_edgelist(citas, source="source", target="target",create_using=nx.DiGraph())

In [17]:
edges = list(G.edges)
# Como el archivo cora no venia con costos, le dimos costos random entre 1 y 15.
# Cualquier cosa se puede revisar la lista costs
costs = [randint(1, 15) for i in range(len(edges))]
edges_costo = []
for i in range(len(edges)):
  edges_costo.append((edges[i][0], edges[i][1], costs[i]))

In [18]:
source_node = 35
edges_rdd = sc.parallelize(edges_costo)
nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct()

In [19]:
current_costs = nodes.map(initialize)

In [20]:
current_costs_dict = dict(current_costs.collect())

In [21]:
def SSSP(max_iterations, convergence, current_costs):
  no_change = 0
  actual_iter = 0
  while no_change < convergence and actual_iter < max_iterations:
    previous_costs = current_costs.collectAsMap()
    messages = edges_rdd.map(lambda x: generate_messages_2(x))
    current_costs = current_costs.union(messages).reduceByKey(lambda x, y: min(x, y))
    current_costs_dict = dict(current_costs.collect())

    if previous_costs == current_costs.collectAsMap():
      no_change += 1
    else:
      no_change = 0

    actual_iter += 1
  print(f'El algoritmo convergio en {actual_iter} iteraciones')
  return current_costs

In [22]:
costos_finales = SSSP(15, 2, current_costs)

El algoritmo convergio en 3 iteraciones


#### Con el ejemplo del grafo de "cora" puede que el nodo inicial no este conectado a todos los nodos por lo que solo entrega los costos de los nodos a los que se pueden llegar. Por esto mismo los filtramos:

In [23]:
final = costos_finales.filter(lambda x: x[1] != float('inf'))
final.collect()

[(82920, 5), (210872, 2), (35, 0), (210871, 7)]

#### Para poder probar el codigo con otros grafos hicimos una funcion para crear grafos que esten bien conectados.

In [24]:
import random

def generate_edges(num_nodes, cost_range):
    edges = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                cost = random.randint(cost_range[0], cost_range[1])
                edges.append((i+1, j+1, cost))
    return edges

# Ejemplo de uso
num_nodes = 20
cost_range = (1, 10)
edges = generate_edges(num_nodes, cost_range)

In [25]:
source_node = 3
edges_rdd = sc.parallelize(edges)
nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct()
current_costs = nodes.map(initialize)
current_costs_dict = dict(current_costs.collect())

In [26]:
costos_finales = SSSP(15, 2, current_costs)

El algoritmo convergio en 3 iteraciones


In [27]:
print("Los costos finales son:\n")
costos_finales.collect()

Los costos finales son:



[(8, 6),
 (16, 4),
 (1, 10),
 (9, 6),
 (17, 7),
 (18, 6),
 (2, 7),
 (10, 3),
 (19, 7),
 (3, 0),
 (11, 5),
 (12, 10),
 (20, 8),
 (4, 3),
 (13, 10),
 (5, 1),
 (6, 2),
 (14, 3),
 (7, 4),
 (15, 2)]